In [1]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display

from package.resources.run import generate_data
from package.resources.utility import (
    createFolder,
    save_object,
    load_object,
    produce_name_datetime
)

from package.plotting_data.single_experiment_plot import *
import matplotlib.pyplot as plt


WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


In [2]:
# Paste or import your 'base_params' dictionary here
base_params = {
    "seed_repetitions": 64,
    "duration_burn_in_research": 0,
    "duration_burn_in": 180,
    "duration_calibration": 276,
    "duration_future": 0,
    "save_timeseries_data_state": 1,
    "compression_factor_state": 1,
    "seed": 5,
    "seed_inputs": 22,
    "ev_research_start_time":0,
    "ev_production_start_time": 0,
    "EV_rebate_state": 0,
    "parameters_rebate_calibration":{
        "start_time": 108,
        "rebate": 10000,
        "used_rebate": 1000
    },
    "parameters_scenarios":{
            "Gas_price": 1,
            "Electricity_price": 1,
            "Grid_emissions_intensity": 0.1
    },
    "parameters_policies":{
        "States":{
            "Carbon_price": 0,
            "Electricity_subsidy": 0,
            "Adoption_subsidy": 0,
            "Adoption_subsidy_used": 0,
            "Production_subsidy": 0,
            "Research_subsidy": 0
        },
        "Values":{
            "Carbon_price":{
                "Carbon_price_init": 0,
                "Carbon_price": 0.2,
                "Carbon_price_state": "flat"
            },
            "Electricity_subsidy":0.5,
            "Adoption_subsidy":10000,
            "Adoption_subsidy_used":20000,
            "Production_subsidy":10000,
            "Research_subsidy":1000000
        }
    },
    "parameters_second_hand":{
        "age_limit_second_hand": 36,
        "max_num_cars_prop": 0.1,
        "burn_in_second_hand_market": 12,
        "scrap_price": 669.8
    },
    "parameters_ICE":{
        "prop_explore": 0.1,
        "N": 15,
        "K": 3,
        "A": 3,
        "rho":[1,0,0],
        "production_emissions":10000,
        "delta": 0.00175,
        "delta_P": 0.0116,
        "transportType": 2,
        "mean_Price": 60000,
        "min_Efficiency": 0.79,
        "max_Efficiency": 3.09,
        "min_Quality": 0,
        "max_Quality": 1,
        "fuel_tank": 469.4,
        "min_Cost": 5000,
        "max_Cost": 70000
    }, 
    "parameters_EV":{
        "prop_explore": 0.1,
        "N": 15,
        "K": 3,
        "A": 4,
        "rho":[1,0,0,0.5],
        "delta_P":0.0087,
        "production_emissions":14000,
        "transportType": 3,
        "min_Efficiency": 2.73,
        "max_Efficiency": 9.73,
        "min_Battery_size": 0,
        "max_Battery_size": 150,
        "min_Cost": 5000,
        "max_Cost": 70000
    },
    "parameters_firm_manager": {
        "J": 16,
        "init_car_age_mean": 0,
        "init_car_age_std": 0,
        "time_steps_tracking_market_data":12,
        "gamma_threshold_percentile": 50,
        "num_beta_segments": 8,
        "num_gamma_segments": 2
    },
    "parameters_firm":{
        "lambda": 1e-3,
        "memory_cap": 30,
        "prob_innovate": 0.083,
        "prob_change_production": 0.083,
        "init_price_multiplier": 1.2,
        "min_profit": 0,
        "max_cars_prod": 10
    },
    "parameters_social_network":{
        "num_individuals":3000,
        "chi_max": 0.9,
        "a_chi": 1.1968, 
        "b_chi": 2.6805,
        "SW_network_density": 0.05,
        "SW_prob_rewire": 0.1,
        "WTP_E_mean": 46646.65434,
        "WTP_E_sd": 39160.31118,
        "nu": 1174,
        "gamma_epsilon": 1e-5,
        "nu_epsilon": 1e-5,
        "prob_switch_car":0.083,
        "proportion_zero_target": 0.005,
        "income_mu":11.225,
        "income_sigma":0.927
    },
    "parameters_vehicle_user":{
        "kappa": 2.25e-4,
        "U_segments_init": 0,
        "W_calibration":1e5,
        "min_W": 1e-10,
        "r": 0.00407412378,
        "mu": 1,
        "alpha": 0.5,
        "zeta":0.29697
    }
}


In [3]:
def run_simulation(base_params):
    fileName = produce_name_datetime("single_experiment")
    print("Saving to:", fileName)

    controller = generate_data(base_params, print_simu=1)
    createFolder(fileName)
    save_object(controller, fileName + "/Data", "controller")
    save_object(base_params, fileName + "/Data", "base_params")
    
    return fileName, controller

fileName, controller = run_simulation(base_params)
print("Simulation complete!")


Saving to: results/single_experiment_22_07_10__12_05_2025
SIMULATION time taken: 0.5614091436068217 minutes or 33.6845486164093 s
Simulation complete!


In [ ]:
def generate_plots(fileName):
    try:
        base_params = load_object(fileName + "/Data", "base_params")
        data_controller = load_object(fileName + "/Data", "controller")
    except FileNotFoundError:
        print("Data files not found.")
        return
    
    dpi = 200

    social_network = data_controller.social_network
    firm_manager = data_controller.firm_manager
    time_series = np.arange(0, len(data_controller.time_series) - base_params["duration_burn_in"])

    calibration_data_output = load_object( "package/calibration_data", "calibration_data_output")
    EV_stock_prop_2010_23 = calibration_data_output["EV Prop"]

    plot_ev_consider_adoption_bought_rate(base_params, social_network,firm_manager, time_series, fileName, EV_stock_prop_2010_23, dpi)
    plot_total_utility(base_params,social_network, time_series, fileName, dpi)
    plot_history_count_buy_ratio(base_params, social_network, fileName, dpi, annotation_height_prop=[0.5, 0.5, 0.5])
    plot_history_car_age(base_params, social_network, time_series,fileName, dpi)
    emissions_decomposed_flow(base_params,social_network, time_series, fileName, dpi)
    plot_history_median_price_by_type(base_params, social_network, fileName, dpi)
    plot_history_mean_price_by_type(base_params, social_network, fileName, dpi)
    plot_price_history(base_params, firm_manager, time_series, fileName, dpi)
    plot_preferences(social_network, fileName, dpi)
    plot_market_concentration_yearly(base_params,firm_manager, time_series, fileName, dpi)
    plot_battery(base_params, firm_manager,social_network,time_series,  fileName, dpi)
    plot_vehicle_attribute_time_series_by_type_split(base_params, social_network, time_series, fileName, dpi)
    plot_prod_vehicle_attribute_time_series_by_type_split(base_params, firm_manager, time_series, fileName, dpi)
    plot_transport_users_stacked(base_params, social_network, time_series, fileName, dpi)
    plot_history_count_buy_stacked(base_params, social_network, fileName, dpi)
    plot_history_count_buy_lines(base_params, social_network, fileName, dpi)
    plot_profit_margins_by_type(base_params, firm_manager, time_series,  fileName)
    plot_total_profit(base_params,firm_manager, time_series, fileName, dpi)
    plot_car_sale_prop(base_params,social_network, time_series, fileName, dpi)

    plt.show()

generate_plots(fileName)
